# Navier Stokes Equation Reduced Basis Method 
Incompressible timedependet Navier Stokes equation

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align}

In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)
# viscosity
nu = 0.001
dt = 0.001

T = 1
dn = int(T/dt)
z = 100  # Number of snapshots

In [3]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet", dgjumps=True)
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [4]:
gfu_N = GridFunction(X)
gfu = GridFunction(X)
velocity = gfu.components[0]


uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Draw(gfu.components[0],mesh,"gfu",sd=3)

In [5]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

In [6]:
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)

mstar += InnerProduct(u,v)*dx + dt*stokes

mstar.Assemble()

mh = BilinearForm(X)

mh += InnerProduct(u,v)*dx
mh.Assemble()

inv = mstar.mat.Inverse(X.FreeDofs())


conv = LinearForm(X)
conv += InnerProduct(grad(velocity)*velocity,v)*dx

In [7]:
rows,cols,vals = a.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))
Fh = f.vec
rows,cols,vals = mh.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))

In [9]:
t = 0
tend = 0
len_h = len(gfu.components[0].vec)
erg_h = np.zeros([len_h,int(dn)+1])

V_POD = np.zeros([len_h,z]) # creat the orthogonal transformationsmatrix for the reducebasis method
line = np.linspace(0,dn-1,z,dtype = int) # stors the number of linear distributed number of iteration from the solution
S_POD = np.zeros([len_h,z])
#line = np.linspace(0,z,z) # stors the number of linear distributed number of iteration from the solution
line

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 111, 121,
       131, 141, 151, 161, 171, 181, 191, 201, 211, 222, 232, 242, 252,
       262, 272, 282, 292, 302, 312, 322, 333, 343, 353, 363, 373, 383,
       393, 403, 413, 423, 433, 444, 454, 464, 474, 484, 494, 504, 514,
       524, 534, 544, 555, 565, 575, 585, 595, 605, 615, 625, 635, 645,
       655, 666, 676, 686, 696, 706, 716, 726, 736, 746, 756, 766, 777,
       787, 797, 807, 817, 827, 837, 847, 857, 867, 877, 888, 898, 908,
       918, 928, 938, 948, 958, 968, 978, 988, 999])

$
M\bigg(\frac{u^{n+1}-u^n}{k}\bigg) + A u^{n+1} +C u^{n} = 0
$

$
M^*\bigg(u^{n+1}-u^n\bigg) = -k A u^n - k Cu^n
$

$
M^* = M + kA 
$

In [10]:
# implicit Euler/explicit Euler splitting method:
erg_h[:,0] = gfu.components[0].vec
S_POD[:,0] = gfu.components[0].vec
o = 1
for i in range(int(dn)):
    conv.Assemble()
    res.data = a.mat * gfu.vec + conv.vec
    gfu.vec.data -= dt * inv * res
    
    for r in line:
        if int(r) == i: #desided which solution will be orthogonalirsed
            if i > 0:
                S_POD[:,o] = gfu.components[0].vec
                o = o + 1
    erg_h[:,i+1] = gfu.components[0].vec #stores the solution for comparing later
    t = t + dt
    Redraw()

orthogonaliese:  10
orthogonaliese:  20
orthogonaliese:  30
orthogonaliese:  40
orthogonaliese:  50
orthogonaliese:  60
orthogonaliese:  70
orthogonaliese:  80
orthogonaliese:  90
orthogonaliese:  100
orthogonaliese:  111
orthogonaliese:  121
orthogonaliese:  131
orthogonaliese:  141
orthogonaliese:  151
orthogonaliese:  161
orthogonaliese:  171
orthogonaliese:  181
orthogonaliese:  191
orthogonaliese:  201
orthogonaliese:  211
orthogonaliese:  222
orthogonaliese:  232
orthogonaliese:  242
orthogonaliese:  252
orthogonaliese:  262
orthogonaliese:  272
orthogonaliese:  282
orthogonaliese:  292
orthogonaliese:  302
orthogonaliese:  312
orthogonaliese:  322
orthogonaliese:  333
orthogonaliese:  343
orthogonaliese:  353
orthogonaliese:  363
orthogonaliese:  373
orthogonaliese:  383
orthogonaliese:  393
orthogonaliese:  403
orthogonaliese:  413
orthogonaliese:  423
orthogonaliese:  433
orthogonaliese:  444
orthogonaliese:  454
orthogonaliese:  464
orthogonaliese:  474
orthogonaliese:  484
o

## POD method
A other way to get the transforamtions matrix is the POD method.
gifen $\mathbb{S} = \{u_h^0,u_h^1,...u_h^{S-1}\}$ where $S$ ist the number of snapshots.

$
C = \mathbb{S}^T\mathbb{S}\\
$

$
C \in \mathbb{R}^{S \times S}
$

decompose the solution $u_h$ into the mean part $\bar{u}_h$ and the fluctuation part $\hat{u}_h$

$\bar{u}_h = \frac{1}{S} \sum_{n=0}^{S-1}u_h^n$  $\,\,\,\,\,\, \bar{u}_h \in \mathbb{R}^{Nh \times S-1}$????

and solve the eigenwalueproblem

$
C\Omega = \Omega\Lambda\\
\Lambda = diag[\lambda_0,...,\lambda_{S-1}], \Omega = [\omega^0,...,\omega^{S-1}]
$

$\lambda_i$ is the $i$th eigenvalue and $\omega^i$ is the corresponding normalized $i$th eigenvector

For a integer $r<<S$ given the $r$th POD basis like:

$
\phi_j = \frac{1}{\sqrt{\lambda_j}}\sum_{n=0}^{r-1}\omega^j_nu_h^n, \,\,\,\,\,\,\,\, j = 0,...,r-1
$

Frage:
For $j=0$  and $r = 3$ the expression becomes to:

$
\phi_0 = \frac{1}{\sqrt{\lambda_0}}\bigg[\omega_0^0 \cdot u_h^0 + \omega_1^0 \cdot u_h^1 +\omega_2^0 \cdot u_h^2\bigg]
$

So $\phi \in \mathbb{R}^{N_h\times r}$

To get a orthogonal basis $V = \{\phi_0,...\phi_{r-1}\}$

In [11]:
ubar = np.zeros(len_h)
#splitting the snapshotsolution like in the paper
for i in range(S_POD.shape[1]):
    ubar = ubar + S_POD[:,i]                   
ubar = 1/S_POD.shape[1] * ubar 
for i in range(S_POD.shape[1]):
    S_POD[:,i] = S_POD[:,i] - ubar

In [12]:
print(ubar)

[ 1.24765049e-16  0.00000000e+00  0.00000000e+00 ... -1.05922749e-01
  2.27022742e-02 -3.49115782e-01]


## POD_Method paper

In [ ]:
C = np.transpose(S_POD).dot(S_POD)
lam,omg = np.linalg.eigh(C)

In [ ]:
lam = np.abs(lam)

In [ ]:
r = 6
V_POD = np.zeros([len_h,r])
for i in range(r):
    for j in range(r):
        V_POD[:,i] += omg[j,i]*S_POD[:,j]
    V_POD[:,i] = 1/np.sqrt(lam[i]) * V_POD[:,i]

In [ ]:
print(V_POD[:,0].dot(V_POD[:,0]))
print(V_POD.shape)

## POD_Method RedBasis Book

In [13]:
def calcPOD(S_POD,r):
    C = np.transpose(S_POD).dot(S_POD)
    V_POD = np.zeros([S_POD.shape[0],r])
    U,Sig,VH = np.linalg.svd(C)
    VH = np.transpose(VH)
    line = np.linspace(0,S_POD.shape[1]-1,r,dtype = int)
    print(line)
    o = 0
    for i in line:
        V_POD[:,o] = 1/(np.sqrt(Sig[i])) * S_POD.dot(VH[:,i])
        o = o + 1
    
    return V_POD

In [14]:
r = 6
V_POD = calcPOD(S_POD,r)

[ 0 19 39 59 79 99]


In [ ]:
r = 6
C = np.transpose(S_POD).dot(S_POD)
V_POD = np.zeros([len_h,r])
U,Sig,VH = np.linalg.svd(C)
VH = np.transpose(VH)
for i in range(r):
    V_POD[:,i] = 1/(np.sqrt(Sig[i])) * S_POD.dot(VH[:,i])


In [ ]:
V_POD[:,0].dot(V_POD[:,0])

# Offline decomposition

$C_h(\omega,u,v) = \int_K(\nabla \omega \cdot u): v$

$C0_j = C_h(\bar{u}_h,\bar{u}_h,\phi_j)$

$C1_{i,j} = C_h(\bar{u}_h,\phi_i,\phi_j) + C_h(\phi_i,\bar{u}_h,\phi_j)$

$C_{i,j,k} = C_h(\phi_i,\phi_j,\phi_k)$




In [21]:
def Ch_ngsolve(wh,uh,vh):
    Ch = 0
    u_N = GridFunction(V)
    v_N = GridFunction(V)
    w_N = GridFunction(V)
    w_N.vec.FV().NumPy()[:] = wh
    u_N.vec.FV().NumPy()[:] = uh
    v_N.vec.FV().NumPy()[:] = vh
    mean_dudu = 0.5 * (Grad(u_N)+Grad(u_N.Other()))
    #jump_u_N = u_N-u_N.Other()
    #mean_uu = 0.5 * (u_N+u_N.Other())
    #n = specialcf.normal(mesh.dim)
    #print(mean_uu)
        
    Ch += Integrate(InnerProduct(Grad(w_N)*u_N,v_N),mesh)
    #Ch += -Integrate(InnerProduct(mean_uu,n),mesh)
    
    return Ch

In [22]:
C0 = np.zeros(r)
w = ubar
u = w 
for i in range(r):
    v = V_POD[:,i]
    C0[i] = Ch_ngsolve(w,u,v)
C0

AttributeError: 'ngsolve.fem.CoefficientFunction' object has no attribute 'derivname'

In [ ]:
C0

In [ ]:
C = np.zeros([r,r,r])
for i in range(r):
    w = V_POD[:,i]
    for j in range(r):
        u = V_POD[:,j]
        for k in range(r):
            v = V_POD[:,k]
            C[i,j,k] = Ch_ngsolve(w,u,v)

In [ ]:
C1 = np.zeros([r,r])
for i in range(r):
    w = ubar
    u = V_POD[:,i]
    for j in range(r):
        v = V_POD[:,j]
        C1[i,j] = Ch_ngsolve(w,u,v)
        C1[i,j] += Ch_ngsolve(u,w,v)
    

In [ ]:
def Bh_ngsolve(uh,vh):
    Bh = 0
    n = specialcf.normal(mesh.dim)
    u_N = GridFunction(V)
    v_N = GridFunction(V)
    u_N.vec.FV().NumPy()[:] = uh
    v_N.vec.FV().NumPy()[:] = vh
    jump_u_N = u_N-u_N.Other()
    
    Bh += Integrate(InnerProduct(Grad(u_N),Grad(v_N)),mesh) 
    
    return Bh

In [ ]:
B0 = np.zeros(r)
u = ubar 
for i in range(r):
    v = V_POD[:,i]
    B0[i] = Bh_ngsolve(u,v)

In [ ]:
B1 = np.zeros([r,r])
for i in range(r):
    u = V_POD[:,i]
    for j in range(r):
        v = V_POD[:,j]
        B1[i,j] = Bh_ngsolve(u,v)

In [ ]:
B1

In [ ]:
def Mh_ngsolve(uh,vh):
    Mh = 0
    u_N = GridFunction(V)
    v_N = GridFunction(V)
    u_N.vec.FV().NumPy()[:] = uh
    v_N.vec.FV().NumPy()[:] = vh
    
    Mh += Integrate(InnerProduct(Grad(u_N),Grad(v_N)),mesh)
    
    return Mh

In [ ]:
M1 = np.zeros([r,r])
for i in range(r):
    u = V_POD[:,i]
    for j in range(r):
        v = V_POD[:,j]
        M1[i,j] = Mh_ngsolve(u,v)

In [ ]:
M1

## Offline decomposition

$B_h^{dg}(u,v) = \sum_{T}\big(\int_T \nabla u : \nabla v dx \big) - \int_{\partial T} (\{\{\nabla u \}\}n) \cdot tang(v) ds - \int_{\partial T} (\{\{\nabla v \}\}n) \cdot tang(u) ds + \int_{\partial T} \frac{4k^2}{h}tang([u]) \cdot tang([v]) ds$

$B0_j = B_h^{dg}(\bar{u}_h,\phi_j)$

$B1_{i,j} = B_h^{dg}(\phi_i,\phi_j)$

In [15]:
def tang(vec):
    return vec - (vec*n)*n

In [27]:
u_N = GridFunction(X)
v_N = GridFunction(X)

u = u_N.components[0]
v = v_N.components[0]

#u_N.vec.FV().NumPy()[:] = 0
#v_N.vec.FV().NumPy()[:] = 0
    
test = GridFunction(V)

In [41]:
u_N = GridFunction(V)
v_N = GridFunction(V)


test = GridFunction(V)

jump_u = u-u.Other()
jump_v = v-v.Other()
n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size
mean_dudu = 0.5 * (grad(u)+grad(u.Other()))
mean_dvdv = 0.5 * (grad(v)+grad(v.Other()))

B_dg = BilinearForm(V)
B_dg += (InnerProduct(Grad(u),Grad(v)))*dx 
B_dg += - (InnerProduct(mean_dudu*n,tang(v)))*ds(skeleton=True)
B_dg += - (InnerProduct(mean_dvdv*n,tang(u)))*ds(skeleton=True)
B_dg += ((4*k*k)/h*(InnerProduct(tang(jump_u),tang(jump_v))))*ds(skeleton=True)

AttributeError: 'ngsolve.fem.CoefficientFunction' object has no attribute 'derivname'

In [83]:
u_N = GridFunction(V)
v_N = V.TestFunction()

jump_u = u_N-u_N.Other()
jump_v = v_N-v_N.Other()
n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

B_hdg = LinearForm(V)
B_hdg += (InnerProduct(Grad(u_N),Grad(v_N)))*dx
B_hdg += -(InnerProduct(Grad(u_N)*n,jump_v))*ds(skeleton=True)
B_hdg += -(InnerProduct(Grad(v_N)*n,jump_u))*ds(skeleton=True)
B_hdg += ((4*k*k*jump_u*jump_v)/h)*ds(skeleton=True)

In [88]:
u_N.vec.FV().NumPy()[:] = V_POD[:,5]
#v_N.vec.FV().NumPy()[:] = V_POD[:,1]
#B_hdg.Apply(u_N.vec,res)
B_hdg.Assemble()

print(Integrate(InnerProduct(B_hdg.vec,B_hdg.vec),mesh))

0.060392607235387766


In [ ]:
u_N = test.vec.CreateVector()
v_N = test.vec.CreateVector()
res = test.vec.CreateVector()

B0 = np.zeros(r)
u_N.vec = V_POD[:,1]
for i in range(r):
    v_N.vec = V_POD[:,i]
    B_dg.Apply(u_N,res)
    #B0[i] = Integrate(InnerProduct(res,res),mesh)
    print(res)
    #B_dg.Assemble()
    #rows,cols,vals = B_dg.mat.COO()
    #Bh = sp.csr_matrix((vals,(rows,cols)))
    #B_dg.Apply(u_N,v_N,res)
    
    
    #B0[i] = Integrate(InnerProduct(res,res),mesh)


In [ ]:
print(Bh)

In [ ]:
B1 = np.zeros([r,r]) 
for i in range(r):
    v_N.vec = V_POD[:,i]
    for j in range(r):
        v_N.vec = V_POD[:,j]
        B_dg.Apply(u_N,v_N)
        B_dg.Assemble()
        rows,cols,vals = B_dg.mat.COO()
        Bh = sp.csr_matrix((vals,(rows,cols)))
        B1[i,j] = norm(Bh,1)


In [ ]:
B1

$C_h(\omega,u,v) = -\frac{1}{2} \bigg(\int_T(\nabla \omega \cdot u): v - \int_{\partial T} (\omega \cdot n)(\{\{u\}\}\cdot v)ds\bigg)$

$C0_j = C_h(\bar{u}_h,\bar{u}_h,\phi_j)$

$C1_{i,j} = C_h(\bar{u}_h,\phi_i,\phi_j) + C_h(\phi_i,\bar{u}_h,\phi_j)$

$C_{i,j,k} = C_h(\phi_i,\phi_j,\phi_k)$

In [23]:
u = V.TrialFunction()
w = V.TrialFunction()
v = V.TrialFunction()
test = GridFunction(V)

jump_u = u-u.Other()
n = specialcf.normal(mesh.dim)
mean_uu = 0.5 * (u+u.Other())


C_dg = BilinearForm(V)
C_dg += (InnerProduct(Grad(w)*u,v))*dx
C_dg += - (InnerProduct(w * n,mean_uu * v))*ds(skeleton=True)


In [ ]:
C_dg.Assemble()
rows,cols,vals = C_dg.mat.COO()
Ch = sp.csr_matrix((vals,(rows,cols)))

In [ ]:
print(Ch)

# RedBasis NavierStokes

In [ ]:
gfu_N = GridFunction(V)
gfu = GridFunction(V)
gfu_N.vec.FV().NumPy()[:] = erg_h[:,0]
gfu.vec.FV().NumPy()[:] = erg_h[:,0]

uN = np.array(gfu_N.vec).dot(V_POD)
velocity_N = gfu_N.components[0]
Draw(gfu_N.components[0],mesh,"gfu_N",sd=3)
Draw(gfu_N.components[0]-gfu.components[0],mesh,"error",sd=3)
err = res.CreateVector()

In [ ]:
dt = 0.0001
dn = 10/dt
Residium = []
time = []
erg_N = np.zeros([len(gfu_N.vec),int(dn)+1])
erg_N[:,0] = gfu_N.vec

I = np.eye(r)
MN_star = M1 + 0.5*dt*nu*B1
inv_N = np.linalg.inv(MN_star)
for i in range(int(dn)):
    #err.vec = sqrt (Integrate ((gfu.components[0]-gfu_N.components[0])*(gfu.components[0]-gfu_N.components[0]),mesh))
    
    res_N = C0 + C1.dot(uN) + uN.dot(C.dot(uN)) + nu*B0 + 0.5*nu*B1.dot(uN)
    uN -= dt * inv_N.dot(res_N)
    #erg_N[:,i+1] = (uN).dot(np.transpose(V_POD))
    
    
    #gfu.vec.FV().NumPy()[:] = erg_h[:,i+1]
    gfu_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V_POD))
    #Residium.append(err.vec)
    #time.append(dt*i)
    Redraw()

In [ ]:
Residium

In [ ]:
M1